In [1]:
from datetime import datetime
import os

from dateutil import parser
from dateutil.relativedelta import relativedelta
import pandas as pd

In [2]:
MUSEUMS_DATABASE_FILENAME = os.path.expanduser('~/Documents/Museums Database.xlsx')

In [3]:
CORRECTIONS = {
    "Alexandre Estrela--February 19-May 8, Vaast Colson & Kati Heck--February 19-May 29":
        "Alexandre Estrela--February 19-May 8; Vaast Colson & Kati Heck--February 19-May 29",
    "João Maria Gusmão & Pedro Paiva-Apr 30-Jul 31":
        "João Maria Gusmão & Pedro Paiva--Apr 30-Jul 31",
    "Yngve Holen-May 13-Aug 7":
        "Yngve Holen--May 13-Aug 7",
    "Manifesta 11-Jun 6-Sept 18":
        "Manifesta 11--Jun 6-Sept 18",
    "Pipilotti Rist Feb-May 8":
        "Pipilotti Rist--Feb-May 8",
    "Esma'/Listen (Lawrence Abu Hamdan etc.) until 21 Aug.":
        "Esma'/Listen (Lawrence Abu Hamdan etc.)--Apr 01-Aug 21",
    "Question the Wall Itself==Nov 19-Mar 26":
        "Question the Wall Itself--Nov 19-Mar 26",
    "Basquiat:\nThe Unknown Notebooks\nFeb 28-May 29":
        "Basquiat: The Unknown Notebooks--Feb 28-May 29",
    "Come As You Are: Art of the 1990s\nFebruary 21 – May 15":
        "Come As You Are: Art of the 1990s--February 21 – May 15",
    "\nDavid Tartakover":
        "David Tartakover",
    "Dan Colen, Helmut Lang, Paola Pivi --Apr 16-Aug\n16 APRIL - AUGUST 2016":
        "Dan Colen, Helmut Lang, Paola Pivi --Apr 16-Aug 31",
    " Rodney\nMcMillian--Mar 24-?":
        "Rodney McMillian--Mar 24-Mar 31",
    "Everything in nature has a lyrical essence, a tragic fate, a comic existence---January 28-March 5":
        "Everything in nature has a lyrical essence, a tragic fate, a comic existence--January 28-March 5",
    "Dominique Gonzalez--Foerster-Apr 23-Aug 7":
        "Dominique Gonzalez-Foerster--Apr 23-Aug 7",
    "Kanon-Fragen-Past Disquiet--March":
        "Kanon-Fragen-Past Disquiet",
    "Kerstin Brätsch--mid-Feb-late May":
        "Kerstin Brätsch--Feb 14-May 31",
    "Fade In--Int. Art Day":
        "Fade In - Int. Art Day",
    "Catherine Opie: Portraits--Jan 30":
        "Catherine Opie: Portraits--Jan 30-Jan 30",
    "Festival of Political Photography--February 19 – May 1":
        "Festival of Political Photography--February 19 - May 1",
    "Come As You Are: Art of the 1990s--February 21 – May 15":
        "Come As You Are: Art of the 1990s--February 21 - May 15",
    "Art Basel Hong Kong--March 24-26, 2016":
        "Art Basel Hong Kong--March 24-Mar 26",
    "Routine Pleasures--May 26-Aug 14 (in LA)":
        "Routine Pleasures--May 26-Aug 14",
    "Cy Twombly--May ?-Feb ?":
        "Cy Twombly--May 15-Feb 14",
    "Alice Neel--Nov 5-?":
        "Alice Neel--Nov 5-Nov 30",
    "Francesca Woodman--Sept 6-Dec ?":
        "Francesca Woodman--Sept 6-Dec 15",
}

In [4]:
non_us_museum_shows = pd.read_excel(MUSEUMS_DATABASE_FILENAME, sheetname="Non-US")
us_museum_shows = pd.read_excel(MUSEUMS_DATABASE_FILENAME, sheetname="US")

In [5]:
museums = []
shows = []

In [6]:
def add_show(museum_name, column_date, show_details):
    if show_details in CORRECTIONS:
        show_details = CORRECTIONS[show_details]

    if '--' in show_details:
        show_name, show_dates = show_details.split('--')
        show_name = show_name.strip()

        start_date_string, end_date_string = show_dates.split('-')
        start_date = parser.parse(
            start_date_string.strip(), default=column_date,
        )
        end_date = parser.parse(
            end_date_string.strip(), default=start_date,
        )
        
        while end_date < start_date:
            end_date += relativedelta(years=1)
    else:
        # No further date details for this show. Use the column header.
        show_name = show_details
        start_date = column_date
        end_date = column_date + relativedelta(months=1, days=-1)

    shows.append({
        'museum_name': museum_name,
        'name': show_name,
        'start_date': start_date,
        'end_date': end_date,
    })

In [7]:
def process_museum_shows(museum_shows):
    for _, row in museum_shows.iterrows():
        museum_name = row.Name

        museums.append({
            'name': museum_name,
            'region': row.Region,
            'country': row.Country,
            'city': row.City
        })

        for key, value in row.iteritems():
            # These aren't the actual shows.
            if key in ('Notes', 'Name', 'Region', 'Country', 'City'):
                continue

            if value != value or value == '?':
                continue

            # It's silly to parse this for every row, but it doesn't matter.
            column_month, column_year = key.split(' \'')
            column_date = parser.parse(
                column_month,
                default=datetime(2000 + int(column_year), 1, 1),
            )
            
            if value in CORRECTIONS:
                value = CORRECTIONS[value]
            
            for show_details in value.split('; '):
                add_show(museum_name, column_date, show_details)

In [8]:
process_museum_shows(non_us_museum_shows)
process_museum_shows(us_museum_shows)

In [9]:
pd.DataFrame(museums, columns=('name', 'region', 'country', 'city'))

,name,region,country,city
0,National Art Museum of China,E. Asia,China,Beijing
1,Kunstsammlung Nordrhein-Westfalen K20,Europe,Germany,Düsseldorf
2,Museum der Moderne Salzburg,Europe,Austria,Salzbug
3,Royal Academy of Arts,Europe,UK,London
4,Tate Modern,Europe,UK,London
5,Tallinn Art Hall,Europe,Estonia,Tallinn
6,Art Gallery of Ontario,N. America,Canada,Toronto
7,Haus der Kunst,Europe,Germany,Munich
8,The Israel Museum,Middle East,Israel,Jerusalem
9,National Gallery of Scotland,Europe,UK,Edinburgh


In [10]:
pd.DataFrame(shows, columns=('museum_name', 'name', 'start_date', 'end_date'))

,museum_name,name,start_date,end_date
0,Kunstsammlung Nordrhein-Westfalen K20,Cyprien Gaillard,2016-01-30,2016-03-20
1,Kunstsammlung Nordrhein-Westfalen K20,Dominique Gonzalez-Foerster,2016-04-23,2016-08-07
2,Kunstsammlung Nordrhein-Westfalen K20,Andreas Gursky,2016-07-02,2016-11-06
3,Museum der Moderne Salzburg,Affichomanie,2016-03-12,2016-07-10
4,Museum der Moderne Salzburg,Anti: Modern - Salzburg,2016-07-01,2016-07-31
5,Royal Academy of Arts,Ann Christopher RA,2016-02-05,2016-05-29
6,Royal Academy of Arts,David Hockney (new work),2016-07-02,2016-10-02
7,Royal Academy of Arts,James Ensor curated by Luc Tuymans,2016-10-29,2017-01-29
8,Tate Modern,Mona Hatoum,2016-05-04,2016-08-21
9,Tate Modern,Bhupen Khakhar,2016-06-01,2016-11-06
